In [1]:
import sqlite3
from pathlib import Path
import pandas as pd

In [2]:
SCHEMA = '''

drop table if exists trip_update;
create table trip_update (
  id integer primary key autoincrement,
  update_timestamp DATETIME not null,
  trip_id int not null
);

drop table if exists trip;
create table trip (
  id integer primary key autoincrement,

  service_id int not null,
  trip_headsign text
);
'''

### Prune DB

In [4]:
conn = sqlite3.connect('prices.db')

In [5]:
conn.executescript(SCHEMA)

In [6]:
conn.commit()

In [7]:
conn.close()

### Insert Data

In [8]:
BASE_PATH = Path('crypto_data/')

In [9]:
files = list(BASE_PATH.glob('*.csv'))

In [11]:
INSERT_STATEMENT = """
INSERT INTO price (
    exchange, symbol, open, high, low, close, volume, day
) VALUES (?, ?, ?, ?, ?, ?, ?, ?);
"""

In [12]:
conn = sqlite3.connect('prices.db')

In [13]:
for file in files:
    exchange, symbol = file.name[:-4].split('_')
    df = pd.read_csv(str(file))
    df['exchange']  = exchange
    df['symbol']  = symbol
    
    values = df[['exchange', 'symbol', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'DateTime']].values
    conn.executemany(INSERT_STATEMENT, values)
    conn.commit()

In [14]:
conn.close()

### Final Test

In [15]:
conn = sqlite3.connect('prices.db')

In [16]:
cursor = conn.cursor()

In [17]:
cursor.execute('SELECT COUNT(*) FROM price;')

In [18]:
cursor.fetchone()

(14421,)

In [19]:
cursor.execute('SELECT * FROM price LIMIT 5;')

In [20]:
cursor.fetchall()

[(1, 'okcoin', 'ltc', 218.5, 233.53, 213.39, 219.965, 6543390, '2018-02-27'),
 (2,
  'okcoin',
  'ltc',
  219.987,
  225.51,
  213.237,
  215.865,
  4382590,
  '2018-02-28'),
 (3,
  'okcoin',
  'ltc',
  216.255,
  227.27700000000002,
  201.976,
  203.09400000000002,
  5222322,
  '2018-03-01'),
 (4,
  'okcoin',
  'ltc',
  203.09400000000002,
  219.238,
  200.207,
  213.87,
  3697634,
  '2018-03-02'),
 (5,
  'okcoin',
  'ltc',
  214.06599999999997,
  217.5,
  205.812,
  214.173,
  3217448,
  '2018-03-03')]

In [21]:
conn.close()

### Exchanges

In [22]:
conn = sqlite3.connect('prices.db')

In [23]:
cursor = conn.cursor()

In [24]:
cursor.execute('SELECT DISTINCT exchange FROM price;')

In [25]:
cursor.fetchall()

[('okcoin',),
 ('hitbtc',),
 ('mexbt',),
 ('kraken',),
 ('okex',),
 ('bittrex',),
 ('bitstamp',),
 ('bitfinex',),
 ('coinbase-pro',),
 ('poloniex',),
 ('cexio',),
 ('huobi',)]

In [26]:
cursor.execute('SELECT DISTINCT symbol FROM price;')

In [27]:
cursor.fetchall()

[('ltc',), ('btc',), ('eth',)]

### Filtered query:

In [28]:
cursor.execute('SELECT * FROM price WHERE symbol = "btc" AND exchange = "bitfinex" AND day = "2019-07-20";')

In [29]:
cursor.fetchall()

[(3396,
  'bitfinex',
  'btc',
  10661,
  10751,
  10115,
  10516,
  387.16246975,
  '2019-07-20')]